In [ ]:
!git clone https://github.com/NtemKenyor/masakhane-pos

Cloning into 'masakhane-pos'...
remote: Enumerating objects: 416, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 416 (delta 12), reused 27 (delta 5), pack-reused 382
Receiving objects: 100% (416/416), 13.46 MiB | 14.65 MiB/s, done.
Resolving deltas: 100% (161/161), done.


In [ ]:
!pip install transformers
!pip install datasets
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.9 MB/s eta 0:00:00


In [ ]:
import os
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from datasets import DatasetDict, Dataset

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback


In [ ]:
class CFG:
  project_name = "Baseline_lacuna_trial_one"
  test_path = "/content/masakhane-pos/data/Test.csv"
  train_path = "/content/masakhane-pos/data/africa_lan.csv"
  model_name = "Davlan/afro-xlmr-mini"
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  seed = 42
  max_length = 38
  valid_languages = ['yor']
  num_classes = None
  num_epochs = 2
  label_mappings = None
  batch_size = 64

In [ ]:
def set_random_seed(random_seed):
  random.seed(random_seed)
  np.random.seed(random_seed)
  os.environ["PYTHONHASHEDSEED"] = str(random_seed)

  torch.manual_seed(random_seed)
  torch.cuda.manual_seed(random_seed)
  torch.cuda.manual_seed_all(random_seed)

  torch.backends.cudnn.deterministic = True

set_random_seed(CFG.seed)
transformers.set_seed(CFG.seed)

In [ ]:
train = pd.read_csv(CFG.train_path)
test = pd.read_csv(CFG.test_path)

display(train.head(),
        test.head())

,word,tag,lang
0,Do,VERB,pcm
1,senator,NOUN,pcm
2,tok,VERB,pcm
3,dis,DET,pcm
4,one,NUM,pcm


,Id,Word,Language,Pos
0,Id00qog2f11n_0,Ne,luo,NaN
1,Id00qog2f11n_1,otim,luo,NaN
2,Id00qog2f11n_2,penj,luo,NaN
3,Id00qog2f11n_3,e,luo,NaN
4,Id00qog2f11n_4,kind,luo,NaN


In [ ]:

train.shape, test.shape


((654842, 3), (32045, 4))

In [ ]:
CFG.num_classes = train.tag.nunique()

In [ ]:
display( 'Language valuecounts',
    train.lang.value_counts(),
    'tag valuecounts',
    train.tag.value_counts(),  )

'Language valuecounts'

ibo    52170
fon    49460
wol    44002
yor    43598
hau    41346
twi    41203
swa    40789
kin    40558
bam    40137
sna    39785
pcm    38570
mos    33792
ewe    28159
xho    25313
lug    24658
nya    24163
zul    24028
bbj    23111
Name: lang, dtype: int64

'tag valuecounts'

NOUN     156687
VERB     111925
PUNCT     65821
PROPN     50323
ADP       49242
PRON      45392
DET       31587
ADJ       27441
AUX       22013
ADV       21229
CCONJ     19867
PART      18440
SCONJ     17556
NUM       15059
X          1523
SYM         511
INTJ        226
Name: tag, dtype: int64

In [ ]:
train = train.rename(columns = {'tag': 'label'})
train = train.dropna()

train['label'] = le.fit_transform(train['label'])
CFG.label_mappings = dict(zip(le.classes_, le.transform(le.classes_)))
print(CFG.label_mappings)


{'ADJ': 0, 'ADP': 1, 'ADV': 2, 'AUX': 3, 'CCONJ': 4, 'DET': 5, 'INTJ': 6, 'NOUN': 7, 'NUM': 8, 'PART': 9, 'PRON': 10, 'PROPN': 11, 'PUNCT': 12, 'SCONJ': 13, 'SYM': 14, 'VERB': 15, 'X': 16}


<ipython-input-10-385f7ed820ff>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['label'] = le.fit_transform(train['label'])


In [ ]:
tokz = AutoTokenizer.from_pretrained(CFG.model_name)
model = AutoModelForSequenceClassification.from_pretrained(CFG.model_name, num_labels = CFG.num_classes).to(CFG.device)


In [ ]:
df_valid = train[train['lang'].isin(CFG.valid_languages)].drop(columns = ['lang'])
df_train = train[train['lang'].isin(CFG.valid_languages)].drop(columns = ['lang'])
test = test.rename(columns = {'Word':'word'})
df_test = test [['word']]

df_train.shape, df_valid.shape, df_test.shape

In [ ]:
df_train.label.mean(), df_valid.label.mean()

In [ ]:
masakhane = DatasetDict({
    "train": Dataset.from_pandas(df_train),
    "valid": Dataset.from_pandas(df_valid),
    "test": Dataset.from_pandas(df_test),
})

masakhane


In [ ]:
def tokenize(x):
  return tokz(x['word'], truncation = True, padding = True, max_length = CFG.max_length)
masakhane_encoded = masakhane.map(tokenize, batched = True, batch_size = CFG.batch_size)

In [ ]:
model_name = f"{CFG.model_name}-masakhane-challenge"
training_args = TrainingArguments(output_dir = model_name,
                                  num_train_epochs = CFG.num_epochs,
                                  learning_rate = 2e-05,
                                  evaluation_strategy ='steps',
                                  save_strategy = 'steps',
                                  save_steps = 10,
                                  eval_steps = 10,
                                  warmup_steps = 10,
                                  metric_for_best_model = "accuracy",
                                  load_best_model_at_end = True,
                                  save_total_limit=1,
                                  logging_strategy="steps",
                                  logging_steps= 10,
                                  per_device_train_batch_size=CFG.batch_size,
                                  per_device_eval_batch_size=CFG.batch_size,
                                  )

In [ ]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  preds = np.argmax(predictions, axis = 1)
  f1_micro = f1_score(labels, preds, average = "micro")
  f1_macro = f1_score(labels, preds, average = "macro")
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc, "f1_micro": f1_micro, "f1_macro": f1_macro}


## Training

In [ ]:
early_stopping_callback = EarlyStoppingCallback(50)
trainer = Trainer(model = model,
                  args = training_args,
                  compute_metrics = compute_metrics,
                  train_dataset = masakhane_encoded['train'],
                  eval_dataset = masakhane_encoded['valid'],
                  tokenizer = tokz,
                  callbacks = [early_stopping_callback],
                  )

trainer.train()


In [ ]:
trainer.evaluate()

Step,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro
10,2.817700,2.586906,0.216259,0.216259,0.025222
20,2.564700,2.436424,0.203944,0.203944,0.035231
30,2.462600,2.384396,0.181238,0.181238,0.029591
40,2.378800,2.322058,0.230080,0.230080,0.041185
50,2.400800,2.283607,0.219145,0.219145,0.023096
60,2.347400,2.319907,0.186467,0.186467,0.046650
70,2.354400,2.253521,0.261253,0.261253,0.066269
80,2.301500,2.243206,0.294508,0.294508,0.084473
90,2.265300,2.217104,0.293371,0.293371,0.081430
100,2.167200,2.054873,0.311107,0.311107,0.110584


{'eval_loss': 0.7859760522842407,
 'eval_accuracy': 0.7637290007362292,
 'eval_f1_micro': 0.7637290007362292,
 'eval_f1_macro': 0.5587162347741503}

In [ ]:
trainer.save_model("baseline_model")

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback


In [ ]:
loaded_model = AutoModelForSequenceClassification.from_pretrained(
    '/content/' + "baseline_model",
    num_labels = CFG.num_classes,

)

loaded_tokenizer = AutoTokenizer.from_pretrained(
    '/content/baseline_model'
)

In [ ]:
test_args = TrainingArguments(
    output_dir= '/content/',
    do_train =False,
    do_predict = True,
    dataloader_drop_last = False
)

trainer = Trainer(
    model = loaded_model,
    args = test_args,
)

test_results = trainer.predict(masakhane_encoded['test'])

In [ ]:
result = test_results.predictions.argmax(axis = 1)
result = pd.DataFrame(result, columns = ['pred'])
result['Id']= test['Id']

name_le_mappings = {v:k for k,v in CFG.label_mappings.items()}
result['pred'] = result['pred'].map(name_le_mappings)
result = result[['Id', 'pred']]
result.to_csv("baseline.csv", index = False)
result.head()

### To
### dos
Improve this approach
Try a token classification approach
